# GOES-18 Satellite Imagery: Utah Region

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/national-data-platform/ep-tutorials/blob/main/examples/goes-utah-satellite.ipynb)

## Why NDP-EP?

This example demonstrates the **real advantage** of using NDP-EP for data management:

- **User A** (first time): Downloads from AWS, processes, registers in NDP-EP
- **User B** (later): Finds processed data in NDP-EP, downloads directly - **much faster!**

### Benefits

| Without NDP-EP | With NDP-EP |
|----------------|-------------|
| Download 57 MB from AWS every time | Download 1 MB from local S3 |
| Process data every time | Already processed |
| ~60 seconds | ~2 seconds |

## Prerequisites

- Access to an NDP-EP API instance
- Valid authentication token

## Setup

In [ ]:
!pip install ndp-ep xarray netCDF4 pyproj folium numpy requests pillow -q

In [ ]:
import numpy as np
import xarray as xr
import requests
import folium
from pyproj import CRS, Transformer
from ndp_ep import Client
import tempfile
import os
import time

## Configuration

In [ ]:
# NDP-EP Configuration
API_URL = "http://localhost:8002"  # Change to your API endpoint
TOKEN = "testing_token"            # Change to your token
ORGANIZATION = "noaa-demo"         # Organization for registering datasets

# Dataset identifiers
DATASET_NAME = "goes18-utah-processed"
BUCKET_NAME = "satellite-data"
S3_KEY = "goes18/utah/goes18_utah_20241125_1800.nc"

# Initialize client
client = Client(base_url=API_URL, token=TOKEN)

# Verify connection
status = client.get_system_status()

print(f"NDP-EP: {status['ep_name']} v{status['api_version']}")
print(f"")
print("Services status:")
print(f"  - Local Catalog: {'✓ Connected' if status.get('backend_connected') else '✗ NOT CONNECTED'}")
print(f"  - S3 Storage:    {'✓ Connected' if status.get('s3_connected') else '✗ NOT CONNECTED'}")

if not status.get('backend_connected'):
    raise RuntimeError("❌ Local catalog is not connected.")
if not status.get('s3_connected'):
    raise RuntimeError("❌ S3 storage is not connected.")

# Ensure organization exists
existing_orgs = client.list_organizations(server="local")
if ORGANIZATION not in existing_orgs:
    print(f"\nCreating organization '{ORGANIZATION}'...")
    client.register_organization({
        "name": ORGANIZATION,
        "title": "NOAA Demo Organization",
        "description": "Demo organization for NOAA satellite data"
    })
    print(f"  ✓ Created")

print("\n✓ Ready!")

In [ ]:
# Create temporary directory for this session
temp_dir = tempfile.mkdtemp()
print(f"Working directory: {temp_dir}")

---

# 🔍 Step 1: Search for Existing Data

Before downloading anything, let's check if someone has already processed GOES-18 data for Utah.

In [ ]:
# Search for existing Utah GOES data
print("Searching for 'GOES utah' in NDP-EP catalog...")
print()

results = client.search_datasets(terms=["goes18", "utah"], server="local")

# Look for our specific processed dataset
found_dataset = None
for dataset in results:
    if dataset.get('name') == DATASET_NAME:
        found_dataset = dataset
        break

if found_dataset:
    print(f"✓ Found existing dataset: {found_dataset['name']}")
    print(f"  Title: {found_dataset.get('title', 'N/A')}")
    print(f"\n→ Skip to USER B section below!")
    DATA_EXISTS = True
else:
    print("✗ No processed GOES-18 Utah data found in catalog.")
    print("\n→ Continue with USER A workflow to download and process.")
    DATA_EXISTS = False

---

# 👤 USER A: First-Time Processing

**Scenario**: No one has processed Utah GOES data yet. We need to:
1. Download from AWS (slow, 57 MB)
2. Process and crop to Utah
3. Upload to NDP-EP S3
4. Register in catalog for others to find

In [ ]:
# Only run if data doesn't exist
if DATA_EXISTS:
    print("⏭ Data already exists. Skip to USER B section.")
else:
    print("="*60)
    print("USER A: Starting full processing workflow")
    print("="*60)
    
    # Start timing
    user_a_start = time.time()

### A.1 Download from AWS

In [ ]:
if not DATA_EXISTS:
    # GOES-18 file on AWS
    GOES_FILE_URL = "https://noaa-goes18.s3.amazonaws.com/ABI-L2-MCMIPC/2024/330/18/OR_ABI-L2-MCMIPC-M6_G18_s20243301801170_e20243301803543_c20243301804085.nc"
    
    print("Downloading from AWS S3 (NOAA public bucket)...")
    print(f"Source: {GOES_FILE_URL[:60]}...")
    print()
    
    download_start = time.time()
    
    goes_file = os.path.join(temp_dir, "goes18_conus.nc")
    response = requests.get(GOES_FILE_URL, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    downloaded_bytes = 0
    with open(goes_file, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
            downloaded_bytes += len(chunk)
            if total_size:
                pct = (downloaded_bytes / total_size) * 100
                print(f"\rDownloading: {downloaded_bytes/1e6:.1f}/{total_size/1e6:.1f} MB ({pct:.0f}%)", end="")
    
    download_time = time.time() - download_start
    download_mb = downloaded_bytes / 1e6
    
    print(f"\n\n✓ Downloaded {download_mb:.1f} MB in {download_time:.1f} seconds")
    print(f"  Speed: {download_mb/download_time:.1f} MB/s")

### A.2 Process: Crop to Utah Region

In [ ]:
if not DATA_EXISTS:
    process_start = time.time()
    
    # Utah geographic bounds
    UTAH_BOUNDS = {
        'lat_min': 37.0, 'lat_max': 42.0,
        'lon_min': -114.0, 'lon_max': -109.0
    }
    
    print("Processing GOES-18 data...")
    print(f"  Cropping to Utah: {UTAH_BOUNDS}")
    
    # Open and read projection info
    goes_ds = xr.open_dataset(goes_file)
    proj = goes_ds['goes_imager_projection']
    h = proj.attrs['perspective_point_height']
    lon_0 = proj.attrs['longitude_of_projection_origin']
    sweep = proj.attrs.get('sweep_angle_axis', 'x')
    
    # Create coordinate transformation
    goes_crs = CRS.from_cf({
        'grid_mapping_name': 'geostationary',
        'longitude_of_projection_origin': lon_0,
        'perspective_point_height': h,
        'sweep_angle_axis': sweep,
        'semi_major_axis': proj.attrs['semi_major_axis'],
        'semi_minor_axis': proj.attrs['semi_minor_axis'],
    })
    transformer = Transformer.from_crs(goes_crs, "EPSG:4326", always_xy=True)
    
    # Transform coordinates
    x = goes_ds['x'].values * h
    y = goes_ds['y'].values * h
    xx, yy = np.meshgrid(x, y)
    lon, lat = transformer.transform(xx, yy)
    
    # Find Utah region
    utah_mask = (
        (lat >= UTAH_BOUNDS['lat_min']) & (lat <= UTAH_BOUNDS['lat_max']) &
        (lon >= UTAH_BOUNDS['lon_min']) & (lon <= UTAH_BOUNDS['lon_max'])
    )
    utah_indices = np.where(utah_mask)
    y_min, y_max = utah_indices[0].min(), utah_indices[0].max()
    x_min, x_max = utah_indices[1].min(), utah_indices[1].max()
    
    # Extract Utah subset
    cmi_utah = goes_ds['CMI_C02'].values[y_min:y_max, x_min:x_max]
    lat_utah = lat[y_min:y_max, x_min:x_max]
    lon_utah = lon[y_min:y_max, x_min:x_max]
    
    # Save as NetCDF
    utah_file = os.path.join(temp_dir, "goes18_utah.nc")
    utah_ds = xr.Dataset({
        'reflectance': (['y', 'x'], cmi_utah),
        'latitude': (['y', 'x'], lat_utah),
        'longitude': (['y', 'x'], lon_utah),
    })
    utah_ds.attrs = {
        'title': 'GOES-18 Utah Region Subset',
        'source': 'NOAA GOES-18 ABI L2 MCMIPC',
        'band': 'C02 (Red, 0.64 µm)',
        'bounds': str(UTAH_BOUNDS),
    }
    utah_ds.to_netcdf(utah_file)
    
    goes_ds.close()
    
    process_time = time.time() - process_start
    processed_mb = os.path.getsize(utah_file) / 1e6
    
    print(f"\n✓ Processing complete in {process_time:.1f} seconds")
    print(f"  Original: {download_mb:.1f} MB → Processed: {processed_mb:.2f} MB")
    print(f"  Reduction: {(1 - processed_mb/download_mb)*100:.0f}%")

### A.3 Upload to NDP-EP S3

In [ ]:
if not DATA_EXISTS:
    print("Uploading processed data to NDP-EP S3...")
    
    # Create bucket
    try:
        client.create_bucket(BUCKET_NAME)
        print(f"  ✓ Created bucket: {BUCKET_NAME}")
    except:
        print(f"  ✓ Bucket exists: {BUCKET_NAME}")
    
    # Upload
    upload_start = time.time()
    with open(utah_file, 'rb') as f:
        client.upload_object(BUCKET_NAME, S3_KEY, f.read(), "application/x-netcdf")
    upload_time = time.time() - upload_start
    
    print(f"  ✓ Uploaded {processed_mb:.2f} MB in {upload_time:.1f} seconds")

### A.4 Register in Catalog

In [ ]:
if not DATA_EXISTS:
    print("Registering dataset in NDP-EP catalog...")
    
    result = client.register_s3_link({
        "resource_name": DATASET_NAME,
        "resource_title": "GOES-18 Utah Region - Visible Band (Processed)",
        "owner_org": ORGANIZATION,
        "s3_bucket": BUCKET_NAME,
        "s3_key": S3_KEY,
        "resource_s3": f"{BUCKET_NAME}/{S3_KEY}",
        "notes": "Pre-processed GOES-18 visible band imagery cropped to Utah region (37-42°N, 109-114°W). Ready for direct visualization."
    })
    
    print(f"  ✓ Registered: {result}")
    
    # Total time for User A
    user_a_total = time.time() - user_a_start
    
    print("\n" + "="*60)
    print("USER A SUMMARY")
    print("="*60)
    print(f"  Download from AWS:  {download_time:.1f}s ({download_mb:.1f} MB)")
    print(f"  Processing:         {process_time:.1f}s")
    print(f"  Upload to NDP-EP:   {upload_time:.1f}s")
    print(f"  ─────────────────────────────")
    print(f"  TOTAL TIME:         {user_a_total:.1f}s")
    print(f"  Data downloaded:    {download_mb:.1f} MB")
    print("="*60)
    
    # Store for comparison
    USER_A_TIME = user_a_total
    USER_A_MB = download_mb

---

# 👤 USER B: Fast Access (Data Already in NDP-EP)

**Scenario**: Another user (or the same user later) needs Utah GOES data.

Instead of repeating all the work, they:
1. Search NDP-EP → Find existing dataset
2. Download processed data directly (only 1 MB!)
3. Visualize immediately

In [ ]:
print("="*60)
print("USER B: Fast workflow (data already in NDP-EP)")
print("="*60)

user_b_start = time.time()

### B.1 Search Catalog

In [ ]:
print("Searching NDP-EP catalog for 'GOES utah'...")

search_start = time.time()
results = client.search_datasets(terms=["goes18", "utah"], server="local")
search_time = time.time() - search_start

# Find our dataset
found = None
for dataset in results:
    if dataset.get('name') == DATASET_NAME:
        found = dataset
        break

if found:
    print(f"\n✓ Found in {search_time:.2f}s: {found['name']}")
    print(f"  Title: {found.get('title', 'N/A')}")
else:
    raise RuntimeError("Dataset not found. Run USER A section first.")

### B.2 Download from NDP-EP (Fast!)

In [ ]:
print("Downloading processed data from NDP-EP S3...")

download_start = time.time()

# Download directly from NDP-EP S3
data = client.download_object(BUCKET_NAME, S3_KEY)

# Save locally
utah_file_b = os.path.join(temp_dir, "goes18_utah_from_ndpep.nc")
with open(utah_file_b, 'wb') as f:
    f.write(data)

download_time_b = time.time() - download_start
download_mb_b = len(data) / 1e6

print(f"\n✓ Downloaded {download_mb_b:.2f} MB in {download_time_b:.2f}s")
print(f"  Speed: {download_mb_b/download_time_b:.1f} MB/s")

### B.3 Load and Visualize (No Processing Needed!)

In [ ]:
print("Loading data for visualization...")

viz_start = time.time()

# Load the pre-processed data
ds_utah = xr.open_dataset(utah_file_b)

cmi_utah = ds_utah['reflectance'].values
lat_utah = ds_utah['latitude'].values
lon_utah = ds_utah['longitude'].values

print(f"  Shape: {cmi_utah.shape}")
print(f"  Reflectance range: {np.nanmin(cmi_utah):.3f} to {np.nanmax(cmi_utah):.3f}")

ds_utah.close()

In [ ]:
# Create visualization
from PIL import Image
import base64

# Normalize and apply gamma correction
cmi_normalized = np.nan_to_num(np.clip(cmi_utah, 0, 1), nan=0)
cmi_uint8 = (np.power(cmi_normalized, 0.5) * 255).astype(np.uint8)

# Save PNG
img = Image.fromarray(cmi_uint8, mode='L')
png_file = os.path.join(temp_dir, "utah_satellite_b.png")
img.save(png_file)

# Create Folium map
bounds = [
    [float(np.nanmin(lat_utah)), float(np.nanmin(lon_utah))],
    [float(np.nanmax(lat_utah)), float(np.nanmax(lon_utah))]
]

m = folium.Map(location=[39.5, -111.5], zoom_start=7, tiles='OpenStreetMap')

with open(png_file, 'rb') as f:
    img_data = base64.b64encode(f.read()).decode()

folium.raster_layers.ImageOverlay(
    image=f"data:image/png;base64,{img_data}",
    bounds=bounds,
    opacity=0.7,
    name="GOES-18 Visible Band"
).add_to(m)

folium.LayerControl().add_to(m)

viz_time = time.time() - viz_start

print(f"\n✓ Visualization ready in {viz_time:.2f}s")

In [ ]:
# Display map
m

In [ ]:
# User B total time
user_b_total = time.time() - user_b_start

print("\n" + "="*60)
print("USER B SUMMARY")
print("="*60)
print(f"  Search catalog:     {search_time:.2f}s")
print(f"  Download from S3:   {download_time_b:.2f}s ({download_mb_b:.2f} MB)")
print(f"  Visualization:      {viz_time:.2f}s")
print(f"  ─────────────────────────────")
print(f"  TOTAL TIME:         {user_b_total:.1f}s")
print(f"  Data downloaded:    {download_mb_b:.2f} MB")
print("="*60)

USER_B_TIME = user_b_total
USER_B_MB = download_mb_b

---

# 📊 Comparison: User A vs User B

In [ ]:
# Get User A values if available, otherwise use typical values
try:
    ua_time = USER_A_TIME
    ua_mb = USER_A_MB
except:
    ua_time = 45.0  # Typical time
    ua_mb = 57.3    # Typical download size

ub_time = USER_B_TIME
ub_mb = USER_B_MB

print("\n" + "="*60)
print("📊 COMPARISON: NDP-EP BENEFITS")
print("="*60)
print()
print(f"                    USER A          USER B")
print(f"                    (first time)    (with NDP-EP)")
print(f"  ──────────────────────────────────────────────")
print(f"  Time:             {ua_time:>6.1f}s         {ub_time:>6.1f}s")
print(f"  Data downloaded:  {ua_mb:>6.1f} MB       {ub_mb:>6.2f} MB")
print(f"  Processing:       Required        None")
print(f"  ──────────────────────────────────────────────")
print()
print(f"  ⏱  Time saved:     {ua_time - ub_time:.1f}s ({(1 - ub_time/ua_time)*100:.0f}% faster)")
print(f"  📦 Data saved:     {ua_mb - ub_mb:.1f} MB ({(1 - ub_mb/ua_mb)*100:.0f}% less)")
print()
print("="*60)

---

## Summary

### Why Use NDP-EP?

**1. Avoid Redundant Work**
- Process data once, use many times
- No need to re-download from slow external sources

**2. Faster Access**
- Local S3 is faster than AWS public buckets
- Pre-processed data is smaller

**3. Data Discovery**
- Search catalog before downloading
- Find datasets others have already prepared

**4. Collaboration**
- Share processed data with your team
- Standard metadata for discovery

## Cleanup (Optional)

In [ ]:
# Uncomment to clean up all resources

# client.delete_resource_by_name(DATASET_NAME)
# client.delete_object(BUCKET_NAME, S3_KEY)
# client.delete_bucket(BUCKET_NAME)

# import shutil
# shutil.rmtree(temp_dir)

print("Done!")